# Multi-Agent Orchestration

In this notebook, you'll create a team of agents that can create detailed research reports.

Plan:

1. Define PMC agent
2. Define writing agents
3. Define lead agents

## 1. Prerequisites

- Python 3.12 or later
- AWS account configured with appropriate permissions
- Access to the Anthropic Claude 3.7 Sonnet model in Amazon Bedrock
- Basic understanding of Python programming

In [ ]:
%pip install -U -r requirements.txt

In [ ]:
import logging

# Sets the logging format and streams logs to stderr
logging.basicConfig(
    level=logging.INFO,
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

# Enables Strands warnings log level
logging.getLogger("strands").setLevel(logging.WARNING)

In [ ]:
MODEL_ID = "global.anthropic.claude-sonnet-4-20250514-v1:0"

## 2. Experimentation

In [ ]:
from strands import Agent
from gather_evidence import gather_evidence_tool

agent = Agent(model=MODEL_ID, tools=[gather_evidence_tool])
response = agent.tool.gather_evidence_tool(pmcid="PMC9438179", question="How safe and effective are GLP-1 drugs for long term use?")

In [ ]:
formatted_response = {
    "toolUseId": response.get('toolUseId'),
    "pmcid": response['content'][1]['json']["pmcid"],
    "citation": response['content'][1]['json']["citation"],
    "question": response['content'][1]['json']["question"],
    "answer": response['content'][0]['text'],
    "evidence": [evidence['context'] for evidence in response['content'][1]['json']['evidence']]
}
formatted_response

### 2.1. DynamoDB stuff

Create table

In [ ]:
import boto3

dynamodb = boto3.resource("dynamodb")

# Check if table exists
table_name = "deep-research-evidence"
try:
    table = dynamodb.Table(table_name)
    table.load()
    print(f"Table '{table_name}' already exists")
except:
    # Create table if it doesn't exist
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[{"AttributeName": "toolUseId", "KeyType": "HASH"}],
        AttributeDefinitions=[
            {"AttributeName": "toolUseId", "AttributeType": "S"},
            {"AttributeName": "pmcid", "AttributeType": "S"},
        ],
        GlobalSecondaryIndexes=[
            {
                "IndexName": "pmcid-index",
                "KeySchema": [{"AttributeName": "pmcid", "KeyType": "HASH"}],
                "Projection": {"ProjectionType": "ALL"},
            }
        ],
        BillingMode="PAY_PER_REQUEST",
    )

    table.wait_until_exists()

Insert example record

In [ ]:
# Example record

example = {
    "toolUseId": "tooluse_gather_evidence_tool_242995659",
    "pmcid": "PMC9438179",
    "citation": 'Nauck, Michael A., and David A. D\'Alessio. "Tirzepatide, a dual GIP/GLP-1 receptor co-agonist for the treatment of type 2 diabetes with unmatched effectiveness regrading glycaemic control and body weight reduction." *Cardiovascular Diabetology*, vol. 21, no. 169, 1 Sept. 2022, doi:10.1186/s12933-022-01604-7.',
    "question": "How safe and effective are GLP-1 drugs for long term use?",
    "answer": "Based on the available evidence, GLP-1 receptor agonists demonstrate favorable safety and efficacy profiles for long-term use. Specifically, tirzepatide, a dual GLP-1/GIP receptor co-agonist, shows sustained glycemic control and weight loss over 52-week treatment periods (Nauck2022 chunk 2). The safety profile appears acceptable, with adverse event rates and cardiovascular outcomes comparable to existing GLP-1 receptor agonist comparators (Nauck2022 chunk 2).\n\nThe long-term effectiveness is characterized by maintained glycemic control and continued weight reduction benefits throughout the 52-week study duration (Nauck2022 chunk 2). From a safety perspective, the cardiovascular outcomes data suggest no increased risk compared to established GLP-1 receptor agonist therapies (Nauck2022 chunk 2).\n\nHowever, the provided context is limited to data from tirzepatide studies over a 52-week period. While this dual agonist represents an advancement in the GLP-1 therapeutic class with unmatched effectiveness regarding glycemic control and body weight reduction, comprehensive long-term safety data beyond one year and broader evidence across the entire class of GLP-1 receptor agonists would be needed for a more complete assessment of long-term safety and efficacy (Nauck2022 chunk 2).",
    "evidence": [
        "In 26‑ to 40‑week trials, GLP‑1 agonists dulaglutide (1.5\u202fmg) and semaglutide (1\u202fmg) lowered HbA1c by ~1.1–1.9% and reduced weight by 2.7–7.8\u202fkg. Gastro‑intestinal events occurred in 30‑40% of patients, with 4‑11% discontinuing, indicating good long‑term efficacy and acceptable safety.",
        "Phase‑2/3 trials of tirzepatide, a GLP‑1/GIP co‑agonist, demonstrate sustained glycaemic control and weight loss over 52‑week periods, with safety data—including adverse‑event rates and cardiovascular outcomes—comparable to GLP‑1RA comparators, indicating effective and acceptable long‑term use.",
        "Tirzepatide, a dual GIP/GLP‑1 agonist, provides robust HbA1c reductions (up to\u202f2.4%), significant weight loss, and a favorable cardiovascular profile (MACE hazard ratios below 1.3, no excess risk). Its 5‑day half‑life supports weekly dosing, and pharmacokinetics are unchanged in renal or hepatic impairment, indicating good long‑term safety and efficacy.",
        "Long‑term GLP‑1 receptor agonists (e.g., semaglutide) consistently lower HbA1c and body weight, improve insulin sensitivity, and reduce appetite. Dual GIP/GLP‑1 agonist tirzepatide shows even greater, sustained reductions in glucose and weight, with no major safety signals reported, though data on GIP‑related effects remain limited.",
        "Clinical data show GLP‑1‑based therapies provide durable glycaemic control, significant weight loss, and improved cardiovascular risk biomarkers over months to years, with a safety profile dominated by transient nausea that can be mitigated (e.g., by GIP co‑agonism).",
    ],
}

import boto3

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table(table_name)

response = table.put_item(
    Item=example
)


Get record

In [ ]:
import boto3

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table(table_name)

response = table.get_item(
    Key={
        'toolUseId': 'tooluse_gather_evidence_tool_242995659'
    }
)
response.get("Item")

Query by secondary index

In [ ]:
import boto3
from boto3.dynamodb.conditions import Key

dynamodb = boto3.resource("dynamodb")
table = dynamodb.Table(table_name)

response = table.query(
    IndexName="pmcid-index", KeyConditionExpression=Key("pmcid").eq("PMC9438179")
)
response.get("Items")

## 3. Define PMC Research Agent

This agent will be similar to the gather evidence agent we created in notebook 2, but with a change in how the evidence is stored. To avoid the "game of telephone" problem, we are going to deterministically store the gathered evidence in a DynamoDB table. This is a big advantage to building AI agents on AWS - they can leverage any of the 200+ services to store and process data.

Let's test out the updated tool to see how it works.

First, we create a DynamoDB table, with `toolUseId` as the primary key and `pmcid` as a global secondary id. This gives us the flexibility to extract evidence records either by tool use or paper. 

In [ ]:
import boto3

dynamodb = boto3.resource("dynamodb")

# Check if table exists
table_name = "deep-research-evidence"
try:
    table = dynamodb.Table(table_name)
    table.load()
    print(f"Table '{table_name}' already exists")
except:
    # Create table if it doesn't exist
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[{"AttributeName": "toolUseId", "KeyType": "HASH"}],
        AttributeDefinitions=[
            {"AttributeName": "toolUseId", "AttributeType": "S"},
            {"AttributeName": "pmcid", "AttributeType": "S"},
        ],
        GlobalSecondaryIndexes=[
            {
                "IndexName": "pmcid-index",
                "KeySchema": [{"AttributeName": "pmcid", "KeyType": "HASH"}],
                "Projection": {"ProjectionType": "ALL"},
            }
        ],
        BillingMode="PAY_PER_REQUEST",
    )

    table.wait_until_exists()

Next, we run our agent with the updated `gather_evidence` tool and pass the db table name as an environment variable.

In [ ]:
from strands import Agent
from search_pmc import search_pmc_tool
from gather_evidence import gather_evidence_tool
import os

MODEL_ID = "global.anthropic.claude-sonnet-4-20250514-v1:0"
QUERY = "How safe and effective are GLP-1 drugs for long term use?"

SYSTEM_PROMPT = """You are a life science research assistant. When given a scientific question, follow this process:

1. Use search_pmc_tool with max_search_result_count between 200 and 500 and max_filtered_result_count between 10 and 20 to find highly-cited papers. Search broadly first, then narrow down. Use temporal filters like "last 5 years"[dp] for recent work. 
2. Identify the PMC ID value for the most relevant paper, then submit the ID and the query to the gather_evidence_tool.
3. Generate a concise answer to the question based on the most relevant evidence, along with PMC ID and URL citations
"""

os.environ['EVIDENCE_TABLE_NAME'] = table_name


# Initialize your agent
pmc_research_agent = Agent(
    system_prompt=SYSTEM_PROMPT,
    tools=[search_pmc_tool, gather_evidence_tool],
    model=MODEL_ID,
)

# Send a message to the agent
response = pmc_research_agent(QUERY)

Let's look at the new records in our db table

In [ ]:
dynamodb = boto3.resource("dynamodb")

# Check if table exists
table_name = "deep-research-evidence"
table = dynamodb.Table(table_name)

records = [record for record in table.scan().get('Items')]
records

We can also query for a records from a specific PMC ID

In [ ]:
example_pmc_id = records[0]["pmcid"]
print(example_pmc_id)

In [ ]:
import boto3
from boto3.dynamodb.conditions import Key
dynamodb = boto3.resource("dynamodb")
table_name = "deep-research-evidence"
table = dynamodb.Table(table_name)


response = table.query(
    IndexName="pmcid-index", KeyConditionExpression=Key("pmcid").eq(example_pmc_id)
)
example_db_records = response.get('Items')
example_db_records

We'll use our `pmc_research_agent` again later in this notebook

## 4. Define Technical Writing Agent

Next, we'll build our writing agent. This agent will take in a question and citations and use the Anthropic Claude citations API to add citations. Let's test how this works with the Amazon Bedrock InvokeModel API 

Ref: https://docs.claude.com/en/docs/build-with-claude/citations

In [ ]:
import boto3
import json

bedrock_client = boto3.client('bedrock-runtime')

example_request = {
    "anthropic_version": "bedrock-2023-05-31",
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "document",
                    "source": {
                        "type": "text",
                        "media_type": "text/plain",
                        "data": "Based on the available context, specific recommendations for managing gastrointestinal adverse events during GLP-1 receptor agonist (GLP-1 RA) therapy include several key strategies. Management emphasizes comprehensive patient education, initiating treatment with low starting doses, and implementing gradual dose titration to minimize adverse events (Gorgojo2023 chunk 2). Dietary adjustments and ongoing monitoring are also essential components of the management approach (Gorgojo2023 chunk 2).\n\nRegarding clinical experience with long-term safety, GI adverse events occur in 40-85% of patients receiving GLP-1 RAs, but these events are typically mild and transient in nature (Gorgojo2023 chunk 2). The adverse events generally resolve following dose escalation, indicating that patients can adapt to therapy over time (Gorgojo2023 chunk 2). \n\nLong-term safety data demonstrate a favorable profile, with most events classified as non-serious and low rates of permanent treatment discontinuation (Gorgojo2023 chunk 2). This clinical experience supports the overall safety profile of GLP-1 RAs for long-term use (Gorgojo2023 chunk 2). The combination of appropriate management strategies and the generally mild, self-limiting nature of GI adverse events allows most patients to continue therapy successfully over extended periods.",
                    },
                    "title": "PMC9821052",
                    "citations": {"enabled": True},
                },
                {
                    "type": "document",
                    "source": {
                        "type": "text",
                        "media_type": "text/plain",
                        "data": "Long-term GLP-1 receptor agonist therapy demonstrates sustained efficacy in maintaining glycemic control and promoting weight loss (Zheng2024 chunk 55). However, therapeutic effectiveness may plateau over extended treatment periods, with studies indicating this limitation is associated with increased orbitofrontal reward activation, as observed with liraglutide (Zheng2024 chunk 55).\n\nThe adverse event profile is predominantly gastrointestinal, with nausea and vomiting representing the most common side effects (Zheng2024 chunk 55). Serious but rare adverse events include pancreatitis and gallbladder disease (Zheng2024 chunk 55).\n\nSeveral absolute contraindications exist for long-term GLP-1 receptor agonist use. Patients with a personal or family history of medullary thyroid carcinoma should not receive these agents (Zheng2024 chunk 55). Multiple endocrine neoplasia type 2 (MEN 2) also represents a contraindication (Zheng2024 chunk 55). Additionally, severe gastrointestinal disorders preclude the use of these medications (Zheng2024 chunk 55).\n\nThe safety and efficacy profile supports long-term use in appropriate patients, though careful patient selection is essential given the specific contraindications, particularly those related to thyroid malignancy risk and pre-existing gastrointestinal pathology.",
                    },
                    "title": "PMC9821052",
                    "citations": {"enabled": True},
                },                
                {
                    "type": "text",
                    "text": "What is the long-term safety profile and effectiveness of GLP-1 receptor agonists? What are the main adverse events and contraindications for long-term use?",
                },
            ],
        }
    ],
    "max_tokens": 1024,
}

response = bedrock_client.invoke_model(
    modelId="us.anthropic.claude-sonnet-4-20250514-v1:0",
    contentType="application/json",
    accept="application/json",
    body=json.dumps(example_request),
)

response_body = json.loads(response["body"].read())

print(response_body.get('content'))

We can now format the response into a citated output

In [ ]:
def print_cited_response(response_content: dict) -> None:
    citations = []
    for content_item in response_content:
        print(content_item.get("text"), end="")
        for citation in content_item.get("citations", []):
            title = citation.get("document_title")
            if title not in citations:
                citations.append(title)
            print(f" ({citations.index(title)+1})", end="")

    print("\n")
    print("## References")
    for i, title in enumerate(citations, start=1):
        print(f"{i}. https://www.ncbi.nlm.nih.gov/pmc/articles/{title}")
    return None

print_cited_response(response_body.get('content'))

Let's try it with an example record from our db

In [ ]:
# Defined above
example_db_records 

In [ ]:
import boto3
import json

contents = []
for record in example_db_records:
    contents.append(
        {
            "type": "document",
            "source": {
                "type": "content",
                "content": [
                    # In this case, we join all of the evidence items together.
                    {"type": "text", "text": "".join(record.get("evidence"))}
                    # If we wanted to reference individual chunks, we could also do:
                    # {"type": "text", "text": evidence}
                    # for evidence in record.get("evidence")
                ],
            },
            "title": record.get("pmcid"),
            "context": record.get("answer"),
            "citations": {"enabled": True},
        },
    )


contents.append(
    {
        "type": "text",
        "text": "What is the long-term safety profile and effectiveness of GLP-1 receptor agonists? What are the main adverse events and contraindications for long-term use?",
    },
)

print("Bedrock InvokeModel request contents are:\n")
for i in contents:
    print(i)
print()

bedrock_client = boto3.client("bedrock-runtime")

example_request = {
    "anthropic_version": "bedrock-2023-05-31",
    "messages": [{"role": "user", "content": contents}],
    "max_tokens": 1024,
}

response = bedrock_client.invoke_model(
    modelId="us.anthropic.claude-sonnet-4-20250514-v1:0",
    contentType="application/json",
    accept="application/json",
    body=json.dumps(example_request),
)

response_body = json.loads(response["body"].read())

print_cited_response(response_body.get('content'))